# **Set up and install**



## *Set up*

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.6.2
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# wait a bit then test
#!curl -X GET "localhost:9200/"

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch-hadoop/elasticsearch-hadoop-7.6.2.zip
!unzip elasticsearch-hadoop-7.6.2.zip

--2021-12-16 04:38:33--  https://artifacts.elastic.co/downloads/elasticsearch-hadoop/elasticsearch-hadoop-7.6.2.zip
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8930910 (8.5M) [application/zip]
Saving to: ‘elasticsearch-hadoop-7.6.2.zip’

elasticsearch-hadoo 100%[===================>]   8.52M  19.2MB/s    in 0.4s    

2021-12-16 04:38:34 (19.2 MB/s) - ‘elasticsearch-hadoop-7.6.2.zip’ saved [8930910/8930910]

Archive:  elasticsearch-hadoop-7.6.2.zip
   creating: elasticsearch-hadoop-7.6.2/
  inflating: elasticsearch-hadoop-7.6.2/NOTICE.txt  
  inflating: elasticsearch-hadoop-7.6.2/LICENSE.txt  
  inflating: elasticsearch-hadoop-7.6.2/README.md  
   creating: elasticsearch-hadoop-7.6.2/dist/
  inflating: elasticsearch-hadoop-7.6.2/dist/elasticsearch-hadoop-7.6.2.jar  
  inflating: elasticsearch-had

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

^C


In [ ]:
!tar -xzf spark-2.4.5-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'spark-2.4.5-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.extraClassPath", "elasticsearch-hadoop-7.6.2/dist/elasticsearch-spark-20_2.11-7.6.2.jar") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "4g")\
        .config("spark.cores.max", "2")\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!curl -X GET "localhost:9200/"

{
  "name" : "fc181ec523e5",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "q-Zgvl-ASaSxwvoEAKKyiQ",
  "version" : {
    "number" : "7.6.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "ef48eb35cf30adf4db14086e8aabd07ef6fb113f",
    "build_date" : "2020-03-26T06:34:37.794943Z",
    "build_snapshot" : false,
    "lucene_version" : "8.4.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
import json

es_write_conf = { "es.nodes": "localhost", "es.batch.size.entries": "1" ,"es.port" : "9200", "es.resource" : "testindex/testdoc", "es.input.json" : "yes", "es.mapping.id": "doc_id" }

data = [ {'some_key': 'some_value', 'doc_id': 123}, {'some_key': 'some_value', 'doc_id': 456}, {'some_key': 'some_value', 'doc_id': 789} ]
rdd = spark.sparkContext.parallelize(data)

def format_data(x):
    return (x['doc_id'], json.dumps(x))

rdd = rdd.map(lambda x: format_data(x))

# rdd.saveAsNewAPIHadoopFile( \
#     path='-', \
#     outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat", \
#     keyClass="org.apache.hadoop.io.NullWritable", \
#     valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", \
#     conf=es_write_conf)

In [ ]:
rdd.collect()

In [ ]:
rdd.saveAsNewAPIHadoopFile( \
    path='-', \
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat", \
    keyClass="org.apache.hadoop.io.NullWritable", \
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", \
    conf=es_write_conf)

In [ ]:
!curl -X GET 'localhost:9200/testindex/_search'

## *Install sample data*

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
PATH_TO_DATA = "ml-latest-small"

--2021-12-16 04:40:05--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.54MB/s    in 0.2s    

2021-12-16 04:40:06 (4.54 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


## *Install elasticsearch from pypi to use a client instance*

In [ ]:
!pip install elasticsearch

     |████████████████████████████████| 385 kB 5.0 MB/s 


In [ ]:
import time
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}], timeout=60, max_retries=10, retry_on_timeout=True)

In [ ]:
es.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'q-Zgvl-ASaSxwvoEAKKyiQ',
 'name': 'fc181ec523e5',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2020-03-26T06:34:37.794943Z',
  'build_flavor': 'default',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.4.0',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.8.0',
  'number': '7.6.2'}}

# **Insert**

## *Spark insert to ES*

### Load rating data

In [ ]:
ratings = spark.read.csv(PATH_TO_DATA + "/ratings.csv", header=True, inferSchema=True)
ratings.cache()
print("Number of ratings: {}".format(ratings.count()))
print("Sample of ratings:")
ratings.show(5)

Number of ratings: 100836
Sample of ratings:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



### Insert by format("es")

In [ ]:
# write ratings data
current_time = time.time()
ratings.write.format("es").save("ratings")
time_spark_insert_to_es = time.time() - current_time
num_ratings_es = es.count(index="ratings")['count']
num_ratings_df = ratings.count()
# check write went ok
print("Dataframe count: {}".format(num_ratings_df))
print("ES index count:  {}".format(num_ratings_es))
print("Time spark insert to es by format('es'): {}".format(time_spark_insert_to_es))

Dataframe count: 100836
ES index count:  100836
Time spark insert to es by format('es'): 12.113914966583252


In [ ]:
# test things out by retrieving a few rating event documents from Elasticsearch
es.search(index="ratings", q="*", size=3)

### Insert by saveAsNewAPIHadoopFile()

In [ ]:
rddtest2 = ratings.rdd.map(lambda x : json.dumps({'userId': x[0], 'movieId': x[1], 'rating': x[2], 'timestamp': x[3]}))

In [ ]:
rddtest3 = rddtest2.zipWithIndex().map(lambda x: (x[1], x[0]))

In [ ]:
es_write_conf = { "es.nodes": "localhost","es.port" : "9200", "es.resource" : "testindex3/testdoc", "es.input.json" : "yes" }
current_time = time.time()
rddtest3.saveAsNewAPIHadoopFile( \
    path='-', \
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat", \
    keyClass="org.apache.hadoop.io.NullWritable", \
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", \
    conf=es_write_conf)
print("Time spark insert to es by saveAsNewAPIHadoopFIle(): {}".format(time.time() - current_time))

In [ ]:
es.search(index="testindex3", q="*", size=3)

## *Normally insert to ES*

In [ ]:
import pandas as pd
df = pd.read_csv(PATH_TO_DATA + '/ratings.csv')
print(df)

In [ ]:
documents = df.to_dict(orient='records')

In [ ]:
from elasticsearch.helpers import bulk

current_time = time.time()
print(bulk(es, documents, index='test-ratings', doc_type='test-ratings-type', raise_on_error=True))
normally_insert_time = time.time() - current_time
print(normally_insert_time)

In [ ]:
es.search(index="test-ratings", q="*", size=3)

## *Parallel insert*

In [ ]:
from elasticsearch.helpers import parallel_bulk
from collections import deque

current_time = time.time()
deque(parallel_bulk(es, documents, index="ratings-test-parallel", thread_count=2, queue_size=2), maxlen = 0)
print("Time parallel insert: {}".format(time.time() - current_time))

In [ ]:
!curl -XPUT "http://localhost:9200/ratings-test-parallel/_settings" -d '{ "index" : { "max_result_window" : 200000 } }' -H "Content-Type: application/json"

In [ ]:
res = es.search(index="ratings-test-parallel", size=200000)
len(res['hits']['hits'])

# **Query**

## Spark query

In [ ]:
q0 ="""{
  "match": {
      "movieId": 3
      }
}"""

q1 = """
  {
    "match": {
      "rating": 4.0
    }
  }
"""

q = """
  {
    "match_all": {}
  }
"""

### Use newApiHadoopRDD()

In [ ]:
es_read_conf = {
    "es.nodes" : "localhost",
    "es.port" : "9200",
    "es.resource" : "ratings/",
    "es.query" : q0
}
current_time = time.time()
for i in range(1000):
  es_rddtestest = spark.sparkContext.newAPIHadoopRDD(
        inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
        keyClass="org.apache.hadoop.io.NullWritable", 
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
        conf=es_read_conf)
print("Time run newApiHadoopRDD: {}".format(time.time() - current_time))
es_rddtestest.count()

Time run newApiHadoopRDD: 82.85260438919067


52

In [ ]:
es_read_conf = {
    "es.nodes" : "localhost",
    "es.port" : "9200",
    "es.resource" : "ratings/",
    "es.query" : q
}
current_time = time.time()
for i in range(1000):
  es_rdd = spark.sparkContext.newAPIHadoopRDD(
      inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
      keyClass="org.apache.hadoop.io.NullWritable", 
      valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
      conf=es_read_conf)
print("Time run newApiHadoopRDD: {}".format(time.time() - current_time))
es_rdd.count()

Time run newApiHadoopRDD: 85.26038980484009


100836

### Use format("es")

In [ ]:
current_time = time.time()
for i in range(1000):
  spark_dftesttest = spark.read.format("es").option("es.query", q0).load("ratings")
print("Time run format('es'): {}".format(time.time() - current_time))
spark_dftesttest.count()

Time run format('es'): 14.306141138076782


52

In [ ]:
current_time = time.time()
for i in range(1000):
  spark_df = spark.read.format("es").option("es.query", q).load("ratings")
print("Time run format('es'): {}".format(time.time() - current_time))
spark_df.count()

Time run format('es'): 7.898252725601196


100836

## Normally query

In [ ]:
!curl -XPUT "http://localhost:9200/ratings/_settings" -d '{ "index" : { "max_result_window" : 200000 } }' -H "Content-Type: application/json"

{"acknowledged":true}

In [ ]:
current_time = time.time()
for i in range(1000):
  restestesetest = es.search(index="ratings", query={"match": {
        "movieId": 3
        }}, size=52)
print("Time run normally query: {}".format(time.time() - current_time))
len(restestesetest['hits']['hits'])

Time run normally query: 4.378681421279907


52

In [ ]:
current_time = time.time()
for i in range(1000):
  res = es.search(index="ratings", query={"match_all": {}}, size=200000)
print("Time run normally query: {}".format(time.time() - current_time))
len(res['hits']['hits'])

Time run normally query: 1901.0982508659363


100836

## *Multi query*

In [ ]:
import json
import logging
from elasticsearch import exceptions as es_exceptions

def msearch(es_conn, queries):
    request = ''
    for each in queries:
        request += '%s \n' %json.dumps(each)
    return es.msearch(body = request)

queries = []
for i in range(1000):
  queries.append(
    {"index": "ratings"}
  )
  queries.append(
      {"query": {
        "match": {
          "movieId": 3
          }
        },"size": 26818
      }
  )
# queries.append(
#     {"index": "ratings"}
# )
# queries.append(
#     {"query": {
#       "match_all": {}
#       },"size": 200000
#     }
# )
# queries.append(
#     {"index": "ratings"}
# )
# queries.append(
#     {"query": {
#       "match": {
#         "movieId": 3
#         }
#       },"size": 200000
#     }
# )
# queries.append(
#     {"index": "ratings"}
# )
# queries.append(
#   {"query": {
#       "match": {
#         "rating": 4.0
#       }
#     },"size": 200000
#   }
# )
current_time = time.time()
q_results = msearch(es, queries)
print("Test multi queries time: {}".format(time.time() - current_time))

Test multi queries time: 3.807823896408081


In [ ]:
len(q_results["responses"][0]['hits']['hits'])

100836

In [ ]:
len(q_results["responses"][1]['hits']['hits'])

52

In [ ]:
len(q_results["responses"][2]['hits']['hits'])

26818

In [ ]:
current_time = time.time()
res1 = es.search(index="ratings", query={"match_all": {}}, size=200000)
res2 = es.search(index="ratings", query={"match": {"movieId": 3}}, size=200000)
res3 = es.search(index="ratings", query={"match": {"rating": 4.0}}, size=200000)
print("Test queries time: {}".format(time.time() - current_time))

Test queries time: 2.6177937984466553


In [ ]:
len(res1['hits']['hits'])

100836

In [ ]:
len(res2['hits']['hits'])

52

In [ ]:
len(res3['hits']['hits'])

26818